# Experimentation and uplift testing
<br>
<br>
*Extend your analysis from Task 1 to help you identify benchmark stores that allow you to test the impact of the trial store layouts on customer sales.*
<br>
<br>
**Here is your task**<br>
Julia has asked us to evaluate the performance of a store trial which was performed in stores:
<br>
- 77, 86 and 88.


To get started use the QVI_data dataset below or your output from task 1 and consider the monthly sales experience of each store.<br>

This can be broken down by:<br>

- total sales revenue
- total number of customers
- average number of transactions per customer

Create a measure to compare different control stores to each of the trial stores to do this write a function to reduce having to re-do the analysis for each trial store.<br>
Once you have selected your control stores, compare each trial and control pair during the trial period. You want to test if total sales are significantly different in the trial period and if so, check if the driver of change is more purchasing customers or more purchases per customers etc.<br>

# Libraries and Data

In [173]:
#Date data type
from datetime import datetime

#Standard
import numpy as np
import statistics
import pandas as pd
import scipy.stats
import turicreate
import turicreate.aggregate as agg

In [174]:
data = turicreate.SFrame.read_csv("QVI_W_data.csv")

Finished parsing file /home/ingrid_silva/Documents/PORTFOLIO/Projects-Intership/Quantium/task2/QVI_W_data.csv

Parsing completed. Parsed 100 lines in 11.1069 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,str,int,int,int,str,int,float,int,str,str,str,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /home/ingrid_silva/Documents/PORTFOLIO/Projects-Intership/Quantium/task2/QVI_W_data.csv

Parsing completed. Parsed 264834 lines in 7.72578 secs.

In [175]:
#data.column_names()

# Before the TRIAL -:> Fev - Apr/19

Here I took the crowned holidays period that consists of 'NOV - DEZ - JAN' as 'before the trial'.<br>
The objective is to measure the development along both periods.<br>
I desire also to check what were the the drivers that led to the success or unsuccess of the trial in these stores.

## Data Preparing

In [176]:
dt_19_01 = data.filter_by(filter(lambda x: x > 201810 and x <= 201901, data['NEW_DATE']), 'NEW_DATE')
dt_19_01['NEW_DATE'].value_counts()

value,count
201812,22835
201901,22161
201811,21852


In [177]:
new_dt = dt_19_01.groupby(key_column_names='STORE_NBR', 
                      operations={'TOT_SALES': agg.SUM('TOT_SALES'),
                                  'NCUSTOMER': agg.COUNT_DISTINCT('LYLTY_CARD_NBR'),
                                  'NTXN':agg.COUNT_DISTINCT('TXN_ID'),
                                  'PROD_QTY':agg.SUM('PROD_QTY'),
                                  'TOT_MEAN':agg.MEAN('TOT_SALES')
                                 })
new_dt

STORE_NBR,PROD_QTY,TOT_MEAN,NTXN,TOT_SALES,NCUSTOMER
118,766,6.95248041775457,381,2662.8,205
130,772,8.852442159383035,388,3443.599999999999,259
257,802,6.866334164588525,401,2753.3999999999996,212
217,786,8.9470737913486,390,3516.2,244
232,794,7.027707808564232,391,2789.9999999999995,207
49,765,8.834196891191713,384,3409.999999999999,247
13,832,6.862980769230769,412,2854.9999999999995,206
241,658,7.070516717325227,325,2326.1999999999994,175
225,712,7.056179775280899,352,2511.9999999999995,194
14,32,5.882352941176471,17,100.00000000000001,17


## Applying Model to 77 Store

In [178]:
trial_store_77 = new_dt.filter_by([77], 'STORE_NBR')

knn_model_77 = turicreate.nearest_neighbors.create(new_dt,
                                                features=['PROD_QTY','TOT_MEAN','TOT_SALES','NCUSTOMER','NTXN'],
                                                label='STORE_NBR',
                                               distance = turicreate.distances.cosine)

knn_model_77.query(trial_store_77)

Starting brute force nearest neighbors model training.

Validating distance components.

Initializing model data.

Initializing distances.

Done.

...

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.37594     | 30.275ms     |

| Done         |         | 100         | 30.919ms     |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,77,0.0,1
0,38,3.1307667706093056e-05,2
0,163,3.887890081277767e-05,3
0,18,6.392289356937919e-05,4
0,53,7.713102017714579e-05,5


## Applying Model to 86 Store

In [179]:
trial_store_86 = new_dt.filter_by([86], 'STORE_NBR')

knn_model_86 = turicreate.nearest_neighbors.create(new_dt,
                                                features=['PROD_QTY','TOT_MEAN','TOT_SALES','NCUSTOMER','NTXN'],
                                                label='STORE_NBR',
                                               distance = turicreate.distances.cosine)

knn_model_86.query(trial_store_86)

Starting brute force nearest neighbors model training.

Validating distance components.

Initializing model data.

Initializing distances.

Done.

...

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.37594     | 655us        |

| Done         |         | 100         | 1.278ms      |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,86,0.0,1
0,207,1.610748707747689e-07,2
0,208,1.779736067408777e-07,3
0,129,2.811688413206781e-07,4
0,110,7.603157088365009e-07,5


## Applying Model to 88 Store

In [180]:
trial_store_88 = new_dt.filter_by([88], 'STORE_NBR')

knn_model_88 = turicreate.nearest_neighbors.create(new_dt,
                                                features=['PROD_QTY','TOT_MEAN','TOT_SALES','NCUSTOMER','NTXN'],
                                                label='STORE_NBR',
                                               distance = turicreate.distances.cosine)

knn_model_86.query(trial_store_88)

Starting brute force nearest neighbors model training.

Validating distance components.

Initializing model data.

Initializing distances.

Done.

...

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.37594     | 470us        |

| Done         |         | 100         | 898us        |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,88,0.0,1
0,81,5.699697867012077e-07,2
0,199,1.013597848098513e-06,3
0,36,1.4786229715069155e-06,4
0,194,1.586006673215401e-06,5


# During the TRIAL -:> Fev - Apr/19

In [181]:
dt_19_02 = data.filter_by(filter(lambda x: x > 201901 and x <= 201904, data['NEW_DATE']), 'NEW_DATE')

### Functions for hypothesis testing

In [182]:
#FUNCTION -:> BEFORE TRIAL - stores_ab_BF(control_nbr, trial_nbr)

def stores_ab_BF(control, trial):
    """
        Output: a new dataframe organized by month and store number.
        Input: two vars (control | trial)
        control-var which receive the control store nbr
        trial-var which receive the trial store nbr
    """
    store_a = dt_19_01.filter_by(control, 'STORE_NBR')
    store_b = dt_19_01.filter_by(trial, 'STORE_NBR')

    BF_19_A = store_a.groupby(key_column_names=['STORE_NBR','NEW_DATE'], 
                          operations={'TOT_SUM': agg.SUM('TOT_SALES'),
                                      'TOT_MEAN': agg.MEAN('TOT_SALES'),
                                      'NCUST_SUM': agg.SUM('LYLTY_CARD_NBR'),
                                      'NCUST_MEAN': agg.MEAN('LYLTY_CARD_NBR')
                                     })
    BF_19_B = store_b.groupby(key_column_names=['STORE_NBR','NEW_DATE'], 
                          operations={'TOT_SUM': agg.SUM('TOT_SALES'),
                                      'TOT_MEAN': agg.MEAN('TOT_SALES'),
                                      'NCUST_SUM': agg.SUM('LYLTY_CARD_NBR'),
                                      'NCUST_MEAN': agg.MEAN('LYLTY_CARD_NBR')
                                     })
    return BF_19_A, BF_19_B

In [183]:
#FUNCTION -:> ALONG TRIAL - stores_ab_AFT(control_nbr, trial_nbr)
def stores_ab_AFT(control, trial):
    """
        Output: a new dataframe organized by month and store number.
        Input: two vars (control | trial)
        control-var which receive the control store nbr
        trial-var which receive the trial store nbr
    """
    store_a = dt_19_01.filter_by(control, 'STORE_NBR')
    store_b = dt_19_01.filter_by(trial, 'STORE_NBR')

    AFT_19_A = store_a.groupby(key_column_names=['STORE_NBR','NEW_DATE'], 
                          operations={'TOT_SUM': agg.SUM('TOT_SALES'),
                                      'TOT_MEAN': agg.MEAN('TOT_SALES'),
                                      'NCUST_SUM': agg.SUM('LYLTY_CARD_NBR'),
                                      'NCUST_MEAN': agg.MEAN('LYLTY_CARD_NBR')
                                     })
    AFT_19_B = store_b.groupby(key_column_names=['STORE_NBR','NEW_DATE'], 
                          operations={'TOT_SUM': agg.SUM('TOT_SALES'),
                                      'TOT_MEAN': agg.MEAN('TOT_SALES'),
                                      'NCUST_SUM': agg.SUM('LYLTY_CARD_NBR'),
                                      'NCUST_MEAN': agg.MEAN('LYLTY_CARD_NBR')
                                     })
    return AFT_19_A, AFT_19_B

In [184]:
#FUNTION -:> PERCENT_AB - PERCENT(control[col],trial[col]) //Try with tot_sum also//

def PERCENT(control,trial):
    """
        Output: percentage difference between pre-trial and trial mean sales.
        Input: two vars (control | trial)
        control-var which receive the control store nbr
        trial-var which receive the trial store nbr
    """
    nsales = trial.sum()/control.sum()
    BF_AB_SCALE = control*nsales
    PRC = abs((BF_AB_SCALE-trial)/BF_AB_SCALE)
    return PRC

In [212]:
#FUNCTION -:> STAND_TAILS - standard_ab(percent,control[col],trial[col])

def standard_ab(percent,control,trial):
    """
        Output: the result of t-stats.
        --- True: if lies inside the bounds - statistically insignificant
        --- False: if lies outside the bounds - statistically significant
        Input: two vars (percent | control | trial)
        percent-the difference between the control and trial stores along trial.
        control-var control store nbr
        trial-var trial store nbr
    """

    STD = percent.std()
    th_95 = control * (1+STD*2)
    th_5 = control * (1-STD*2)
    print('5th tail' + str(th_5))
    print('95th tail:' + str(th_95))
    print('Trial': + str(trial))

## Testing sales' stores - control and trial

In [213]:
#TRIAL(77) vs 163 - TOT_SALES
#Building datas
BF_163, BF_77 = stores_ab_BF(163, 77)
AFT_163, AFT_77 = stores_ab_AFT(163, 77)

#Percentage - TOT_SUM
perc_16377 = PERCENT(BF_163['TOT_SUM'],BF_77['TOT_SUM'])

#Result - TOT_SUM - [fev - mar - apr]
result_16377 = standard_ab(perc_16377,BF_163['TOT_SUM'],AFT_77['TOT_SUM'])
result_16377

5th tail[218.4226204220254, 185.71762912888792, 174.28756839633732]
95th tail:[305.17737957797465, 259.48237087111215, 243.51243160366275]
Trial Value:204.40000000000003
Trial Value:245.30000000000004
Trial Value:267.30000000000007


array([ True, False, False])

In [214]:
#TRIAL(77) vs 163 - NCUSTOMER
#Percentage - TOT_SUM
perc_16377_customer = PERCENT(BF_163['NCUST_SUM'],BF_77['NCUST_SUM'])

#Result - TOT_SUM
result_16377_customer = standard_ab(perc_16377_customer,BF_163['NCUST_SUM'],AFT_77['NCUST_SUM'])
result_16377_customer

5th tail[1626342.7323926806, 1314800.409012318, 1418765.7056390727]
95th tail:[13718661.26760732, 11090713.590987682, 11967690.294360926]
Trial Value:3012973
Trial Value:19169576
Trial Value:3785902


array([ True,  True,  True])

In [215]:
#TRIAL(86) vs 207 - TOT_SALES

#Building datas
BF_207, BF_86 = stores_ab_BF(207, 86)
AFT_207, AFT_86 = stores_ab_AFT(207, 86)

#Percentage - TOT_SUM
perc_20786 = PERCENT(BF_163['TOT_SUM'],BF_77['TOT_SUM'])

#Result - TOT_SUM - [fev - mar - apr]
result_20786 = standard_ab(perc_20786,BF_163['TOT_SUM'],AFT_86['TOT_SUM'])
result_20786

5th tail[218.4226204220254, 185.71762912888792, 174.28756839633732]
95th tail:[305.17737957797465, 259.48237087111215, 243.51243160366275]
Trial Value:918.0
Trial Value:841.4
Trial Value:841.2


array([False, False, False])

In [216]:
#TRIAL(86) vs 207 - NCUSTOMER

#Percentage - TOT_SUM
perc_20786_customer = PERCENT(BF_207['NCUST_SUM'],BF_86['NCUST_SUM'])

#Result - TOT_SUM
result_20786_customer = standard_ab(perc_20786_customer,BF_207['NCUST_SUM'],AFT_86['NCUST_SUM'])
result_20786_customer

5th tail[27488390.419213854, 24522054.662442274, 23731130.003930666]
95th tail:[30095159.580786142, 26847521.337557722, 25981591.99606933]
Trial Value:11284157
Trial Value:11197109
Trial Value:10335626


array([False, False, False])

In [217]:
#TRIAL(88) VS 81 - TOT_SALES

#Building datas
BF_81, BF_88 = stores_ab_BF(81, 88)
AFT_81, AFT_88 = stores_ab_AFT(81, 88)

#Percentage - TOT_SUM
perc_8188 = PERCENT(BF_81['TOT_SUM'],BF_88['TOT_SUM'])

#Result - TOT_SUM - [fev - mar - apr]
result_8188 = standard_ab(perc_8188,BF_81['TOT_SUM'],AFT_88['TOT_SUM'])
result_8188

5th tail[1099.341039498177, 991.6308725046557, 1281.5907883743532]
95th tail:[1233.8589605018235, 1112.9691274953443, 1438.409211625647]
Trial Value:1325.2
Trial Value:1266.3999999999996
Trial Value:1382.8000000000002


array([False, False,  True])

In [218]:
#TRIAL(88) vs 81 - NCUSTOMER

#Percentage - TOT_SUM
perc_8188_customer = PERCENT(BF_81['NCUST_SUM'],BF_88['NCUST_SUM'])

#Result - TOT_SUM
result_8188_customer = standard_ab(perc_8188_customer,BF_81['NCUST_SUM'],AFT_88['NCUST_SUM'])
result_8188_customer

5th tail[8081027.04320154, 7345314.47466727, 9550852.769056225]
95th tail:[13351682.95679846, 12136119.52533273, 15780167.230943775]
Trial Value:22423013
Trial Value:12875401
Trial Value:13845499


array([False, False,  True])

# Next Step
- Dashboard:
    - Comparison between each pair (control and trial) > tot_sales and ncustomer
    - Plot per month (Fev - Mar - Apr) of each pair